## Analisando Dados com Python - Cancelamento de Clientes

Você foi contratado por uma empresa com mais de 800 mil clientes para um projeto de Dados. Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.

In [2]:
#Importando as bibliotecas necessárias para a análise dos dados e criação do gráfico
import pandas as pd
import plotly.express as px
import plotly.io as pio

import os

In [3]:
# Carregando a base de dados e removendo a coluna CustomerID que não será utilizada
tabela = pd.read_csv("database/cancelamentos_sample.csv")
tabela = tabela.drop("CustomerID", axis=1)
display(tabela)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,23.0,Male,13.0,22.0,2.0,1.0,Standard,Annual,909.58,23.0,0.0
1,49.0,Male,55.0,16.0,3.0,6.0,Premium,Monthly,207.00,29.0,1.0
2,30.0,Male,7.0,1.0,0.0,8.0,Basic,Annual,768.78,7.0,0.0
3,26.0,Male,40.0,5.0,3.0,8.0,Premium,Annual,398.00,12.0,1.0
4,27.0,Female,17.0,30.0,5.0,6.0,Basic,Annual,507.00,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
49995,62.0,Female,35.0,7.0,2.0,8.0,Basic,Annual,232.00,15.0,1.0
49996,36.0,Male,43.0,21.0,2.0,30.0,Basic,Quarterly,928.00,30.0,1.0
49997,55.0,Male,42.0,8.0,1.0,12.0,Basic,Monthly,326.00,27.0,1.0
49998,40.0,Female,14.0,19.0,1.0,17.0,Premium,Quarterly,826.76,12.0,0.0


In [4]:
# Identificando e removendo valores vazios
display(tabela.info())
tabela = tabela.dropna()
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idade                   50000 non-null  float64
 1   sexo                    49997 non-null  object 
 2   tempo_como_cliente      49998 non-null  float64
 3   frequencia_uso          50000 non-null  float64
 4   ligacoes_callcenter     50000 non-null  float64
 5   dias_atraso             50000 non-null  float64
 6   assinatura              50000 non-null  object 
 7   duracao_contrato        50000 non-null  object 
 8   total_gasto             50000 non-null  float64
 9   meses_ultima_interacao  50000 non-null  float64
 10  cancelou                50000 non-null  float64
dtypes: float64(8), object(3)
memory usage: 4.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 49996 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idade                   49996 non-null  float64
 1   sexo                    49996 non-null  object 
 2   tempo_como_cliente      49996 non-null  float64
 3   frequencia_uso          49996 non-null  float64
 4   ligacoes_callcenter     49996 non-null  float64
 5   dias_atraso             49996 non-null  float64
 6   assinatura              49996 non-null  object 
 7   duracao_contrato        49996 non-null  object 
 8   total_gasto             49996 non-null  float64
 9   meses_ultima_interacao  49996 non-null  float64
 10  cancelou                49996 non-null  float64
dtypes: float64(8), object(3)
memory usage: 4.6+ MB


None

In [5]:
# Verificando a quantidade de cancelamentos
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

cancelou
1.0    28393
0.0    21603
Name: count, dtype: int64

cancelou
1.0    56.8%
0.0    43.2%
Name: proportion, dtype: object

In [6]:

display(tabela["duracao_contrato"].value_counts(normalize=True))
display(tabela["duracao_contrato"].value_counts())

duracao_contrato
Annual       0.403152
Quarterly    0.399152
Monthly      0.197696
Name: proportion, dtype: float64

duracao_contrato
Annual       20156
Quarterly    19956
Monthly       9884
Name: count, dtype: int64

In [7]:
# analisando o contrato mensal
display(tabela.groupby("duracao_contrato").mean(numeric_only=True))
# descobrimos aqui que a média de cancelamentos é 1, ou seja, praticamente todos os contratos mensais cancelaram (ou todos)

,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
duracao_contrato,,,,,,,,
Annual,38.783985,31.416452,15.910449,3.277585,12.533985,650.925840,14.231544,0.464080
Monthly,41.428470,30.677964,15.577600,4.923917,15.078814,547.508921,15.392655,1.000000
Quarterly,38.833935,31.522099,15.842504,3.256113,12.480006,654.102443,14.364602,0.458759


In [8]:
# então descobrimos que contrato mensal é ruim, vamos tirar ele e continuar analisando
tabela = tabela[tabela["duracao_contrato"]!="Monthly"]
display(tabela)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,23.0,Male,13.0,22.0,2.0,1.0,Standard,Annual,909.58,23.0,0.0
2,30.0,Male,7.0,1.0,0.0,8.0,Basic,Annual,768.78,7.0,0.0
3,26.0,Male,40.0,5.0,3.0,8.0,Premium,Annual,398.00,12.0,1.0
4,27.0,Female,17.0,30.0,5.0,6.0,Basic,Annual,507.00,15.0,1.0
6,49.0,Male,6.0,7.0,0.0,0.0,Standard,Annual,751.00,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
49994,63.0,Male,16.0,24.0,2.0,18.0,Standard,Quarterly,442.00,26.0,1.0
49995,62.0,Female,35.0,7.0,2.0,8.0,Basic,Annual,232.00,15.0,1.0
49996,36.0,Male,43.0,21.0,2.0,30.0,Basic,Quarterly,928.00,30.0,1.0
49998,40.0,Female,14.0,19.0,1.0,17.0,Premium,Quarterly,826.76,12.0,0.0


cancelou
0.0    21603
1.0    18509
Name: count, dtype: int64

cancelou
0.0    53.9%
1.0    46.1%
Name: proportion, dtype: object

In [9]:
# chegamos agora em menos da metade de pessoas cancelando, mas ainda temos muitas pessoas ai, vamos continuar analisando
display(tabela["assinatura"].value_counts(normalize=True))
display(tabela.groupby("assinatura").mean(numeric_only=True))
# vemos que assinatura é quase 1/3, 1/3, 1/3
# e que os cancelamentos são na média bem parecidos, então fica difícil tirar alguma conclusão da média, vamos precisar ir mais a fundo

assinatura
Standard    0.343264
Premium     0.333466
Basic       0.323270
Name: proportion, dtype: float64

,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
assinatura,,,,,,,,
Basic,38.777049,32.266908,15.938459,3.328064,12.538752,648.339261,14.317036,0.476517
Premium,38.885766,31.120813,15.762111,3.238038,12.430697,653.459842,14.306071,0.453947
Standard,38.764035,31.055850,15.929697,3.237345,12.551601,655.504070,14.271479,0.454499


In [10]:
# vamos criar gráfico, porque só com números tá difícil de visualizar

#for coluna in tabela.columns:
    #grafico = px.histogram(tabela, x=coluna, color="cancelou")
    #grafico.show()
# Para diminuir o tamanho do aaquivo, vamos salvar os gráficos em uma pasta separada
if not os.path.exists('graficos'):
    os.makedirs('graficos')

for coluna in tabela.columns:
    grafico = px.histogram(tabela, x=coluna, color="cancelou")
    # Save the figure as a png image
    pio.write_image(grafico, f'graficos/{coluna}.png')

In [11]:
# com os graficos a gente consegue descobrir muita coisa:
# dias atraso acima de 20 dias, 100% cancela
# ligações call center acima de 5 todo mundo cancela

tabela = tabela[tabela["ligacoes_callcenter"]<5]
tabela = tabela[tabela["dias_atraso"]<=20]
display(tabela)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

# se resolvermos isso, já caímos para 18% de cancelamento
# é claro que 100% é utópico, mas com isso já temos as principais causas (ou talvez 3 das principais):
# - forma de contrato mensal
# - necessidade de ligações no call center
# - atraso no pagamento

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,23.0,Male,13.0,22.0,2.0,1.0,Standard,Annual,909.58,23.0,0.0
2,30.0,Male,7.0,1.0,0.0,8.0,Basic,Annual,768.78,7.0,0.0
3,26.0,Male,40.0,5.0,3.0,8.0,Premium,Annual,398.00,12.0,1.0
6,49.0,Male,6.0,7.0,0.0,0.0,Standard,Annual,751.00,11.0,0.0
7,33.0,Male,15.0,18.0,1.0,20.0,Standard,Annual,839.71,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
49991,24.0,Female,33.0,28.0,1.0,11.0,Premium,Annual,844.05,9.0,0.0
49992,37.0,Male,46.0,29.0,2.0,9.0,Standard,Quarterly,517.86,3.0,0.0
49994,63.0,Male,16.0,24.0,2.0,18.0,Standard,Quarterly,442.00,26.0,1.0
49995,62.0,Female,35.0,7.0,2.0,8.0,Basic,Annual,232.00,15.0,1.0


cancelou
0.0    21446
1.0     4821
Name: count, dtype: int64

cancelou
0.0    81.6%
1.0    18.4%
Name: proportion, dtype: object